In [ ]:
import pandas as pd

In [ ]:
PATH = '../data/lda_data.csv'
TEST = 5
TOPIC = 6

In [ ]:
def split_data(path, test_count):
    df = pd.read_csv(path)
    y = df.sample(n=test_count, random_state=42)
    x = df.drop(y.index)
    return x.reset_index(drop=True), y.reset_index(drop=True)

x, y = split_data(PATH, TEST)

In [ ]:
import pandas as pd
from gensim import corpora, models
from gensim.models import LdaMulticore
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# 1. 문자열 → 토큰 리스트
chunk['tokens'] = chunk['Abstract'].apply(lambda x: x.strip().split())

# 2. 토큰 리스트 모음 → Dictionary & Corpus 생성
tokenized_doc = chunk['tokens'].tolist()
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

# 3. LDA 멀티코어 학습
lda_model = LdaMulticore(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5,         # 원하는 토픽 수
    random_state=42,
    passes=10,            # 학습 반복 횟수
    workers=4,            # CPU 코어 수
    chunksize=100,
    per_word_topics=True
)

# 4. 토픽 출력
for idx, topic in lda_model.print_topics(-1):
    print(f"🟢 Topic {idx}:\n{topic}\n")

# 5. pyLDAvis 시각화
vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.show(vis_data)  # 주피터 환경이면 이걸로 바로 띄움


In [ ]:
import matplotlib.pyplot as plt

# 날짜 형식 변환
chunk['Date'] = pd.to_datetime(chunk['Date']).dt.to_period('M').astype(str)  # 'YYYY-MM'

# ① 문서별 토픽 추정 (가장 확률 높은 토픽 ID)
doc_topics = [max(model[doc], key=lambda x: x[1])[0] for doc in corpus]
chunk['Dominant_Topic'] = doc_topics

# ② Date와 Dominant_Topic 기준으로 문서 수 세기
df_trend = chunk.groupby(['Date', 'Dominant_Topic']).size().unstack(fill_value=0)

# ③ 모든 토픽 추세 (한 프레임에)
plt.figure(figsize=(12, 6))
for topic_id in df_trend.columns:
    plt.plot(df_trend.index, df_trend[topic_id], label=f'Topic {topic_id}')
plt.title('📈 Monthly Topic Trends (All Topics)')
plt.xlabel('Date')
plt.ylabel('Document Count')
plt.legend()
plt.tight_layout()
plt.show()

# ④ 개별 토픽 추세 (하나씩)
for topic_id in df_trend.columns:
    plt.figure(figsize=(8, 4))
    plt.plot(df_trend.index, df_trend[topic_id], marker='o')
    plt.title(f'📊 Trend of Topic {topic_id}')
    plt.xlabel('Date')
    plt.ylabel('Document Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
